In [ ]:
import numpy as np
from agent.OnOff_Agent import OnOffAgent
from environment.ContinuousEnvironment import ContinuousSimpleEnvironment

import pandas as pd

%load_ext autoreload
%autoreload 2

## Set environment 

In [ ]:

from Performance import all_combinations_list
from Performance import search_similar
import Performance
from logger.SimpleLogger import SimpleLogger

results_dict = {}

for alpha_value in [0.25,1,4]:
    env = ContinuousSimpleEnvironment(param_list=['Tair', 'RH', 'Tmrt', 'Tout', 'Qheat', 'Occ'],
    alpha=alpha_value,
    beta=1,
    min_temp=16,
    max_temp=21,
    modelname='CELLS_v1.fmu',
    simulation_path=r'C:\Users\hbenoit\Desktop\DIET_Controller\EnergyPlus_simulations\simple_simulation',
    days=151,
    hours=24,
    minutes=60,
    seconds=60,
    ep_timestep=6)


    num_episodes = 5

    logging_path = r"C:\Users\hbenoit\Desktop\DIET_Controller\logs\simple_simulation"

    agent = OnOffAgent(env=env)

    results_path, summary_df = agent.train(logging_path= logging_path,
             num_iterations= None,
             num_episodes=num_episodes,
             log=True)

    results_dict[alpha_value] = Performance.cumulative_reward(summary_df)


In [ ]:
results_dict

In [ ]:
import Performance

from agent.DDPG_Agent import DDPG_Agent

df = Performance.load_summary_df(r"C:\\Users\\hbenoit\\Desktop\\DIET_Controller\\logs\\simple_simulation\\DDPG_Agent\\results\\2022_6_19\\results_2022_6_19_6_41")


Performance.cumulative_reward(df)

In [ ]:
import Performance

searching_directory = r"C:\Users\hbenoit\Desktop\DIET_Controller\logs\simple_simulation\OnOff_Agent"

conditions = { "alpha":["=",0.25],
    "beta":["=",1], "num_episodes":["=",10]}

best_path_list = Performance.search_paths(
searching_directory,
conditions=conditions,
top_k=1,
utility_function=Performance.cumulative_reward,
normalized=True)

best_path_list

In [ ]:
import Performance
from logger.SimpleLogger import SimpleLogger


for alpha_value in [1]:
    env = ContinuousSimpleEnvironment(param_list=['Tair', 'RH', 'Tmrt', 'Tout', 'Qheat', 'Occ'],
    alpha=alpha_value,
    beta=1,
    min_temp=16,
    max_temp=21,
    modelname='CELLS_v1.fmu',
    simulation_path=r'C:\Users\hbenoit\Desktop\DIET_Controller\EnergyPlus_simulations\simple_simulation',
    days=151,
    hours=24,
    minutes=60,
    seconds=60,
    ep_timestep=6)

    parameter = ("is_step",[True])

    best_agent_path = r"C:\Users\hbenoit\Desktop\DIET_Controller\logs\simple_simulation\OnOff_Agent\results\2022_6_20\results_2022_6_20_2_12"




    logging_path = r"C:\Users\hbenoit\Desktop\DIET_Controller\logs\simple_simulation"

    utility_function = Performance.cumulative_reward
    agent = OnOffAgent(env=env)
    num_episodes = 1
    num_iterations = env.numsteps
    agent_name = "OnOff_Agent"

    results_dict = Performance.across_runs(
    agent=agent,
    agent_config_path=best_agent_path,
    parameter=parameter,
    num_episodes=num_episodes,
    num_iterations=num_iterations,
    utility_function=utility_function,
    alpha=0.05,
    window=6,
    column_names=["Tset","Reward"],
    need_load=False)

    logger = SimpleLogger(
            logging_path=logging_path,
            agent_name="OnOff_Agent",
            num_episodes=num_episodes,
            num_iterations=num_iterations,
        )

    results_dict["alpha"]= agent.env.alpha
    results_dict["beta"]= agent.env.beta

    logger.log_performance_pipeline(results_dict,fixed_policy=False)